[한국전력공사\_공동주택 전력·기상 융합데이터\_20221231 | 공공데이터포털](https://www.data.go.kr/data/15120804/fileData.do)

한국전력공사_공동주택 전력·기상 융합데이터
한국전력공사와 기상청이 공동주택 부하관리관련 연구개발 활성화를 목적으로 제공하는 데이터셋입니다.
데이터셋은 기상청 동네예보 격자단위 시간대별 공동주택 전력부하와 기상관측값을 포함하고 있습니다.
공동주택 전력부하는 각 연도별 결측치가 없는 단지에 한하여 합계한 통계값이며, 격자내 공동주택이 10개 이상인 격자에 대해서만 제공하고 있습니다.

[2024 날씨 빅데이터콘테스트 설명동영상(과제4, 전력분야) - YouTube](https://www.youtube.com/watch?v=9xCBKNrymig&t=265s)


---

**| 학습 데이터(전력)**

-   **제공기간** : 2020년, 2021년, 2022년 (총 3개년)
-   **자료설명** : 원격검침이 이루어지는 공동주택 중 각 연도별 결측치가 없는 단지를 대상으로 기상예보 격자별로 산출된 전력 통계값. 격자내 공동주택이 10개 이상인 격자에 대해서만 공개함.

**컬럼 정보**

| 변수명        | 변수 설명      | 항목설명                                                                                                |
| :------------ | :------------- | :------------------------------------------------------------------------------------------------------ |
| TM            | 날짜           | 공동주택 전력부하 측정 날짜(시간포함, 단위(0~23시))                                                          |
| HH24          | 시간           | 공동주택 전력부하 측정 시간(1~24), 5시는 4시 1분~5시 00분까지의 전력부하를 의미                                  |
| weekday       | 요일           | 요일을 숫자형식으로 표시 월요일(0)~일요일(6)                                                                 |
| week\_name    | 주중 주말      | 주중 주말을 숫자형식으로 표시 주중(0), 주말(1)                                                               |
| SUM\_QCTR     | 계약전력합계   | 해당격자의 전력통계 산출에 포함된 공동주택의 계약전력 합계                                                       |
| N             | 공동주택 수    | 해당격자의 전력통계 산출에 포함된 공동주택의 수, 단위(단지)                                                      |
| SUM\_LOAD     | 전력수요 합계  | 해당격자/시각에 측정된 공동주택의 전력수요 합계                                                                |
| N\_MEAN\_LOAD | 전력부하량 평균 | 격자내 총 전력부하량을 아파트 수로 나누어 격자의 평균 부하량을 산출                                                 |
| elec          | 전력기상지수   | 해당격자의 공동주택의 연평균 부하량을 100으로 했을 때, 해당 시각에 예상되는 부하량을 상대적인 수치로 표현                     |

---

## 전력부하 합계 예측 모델 

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib
from sklearn.model_selection import train_test_split, cross_val_score, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, r2_score
import datetime

In [ ]:
data_folder = 'data'
csv_files = [f for f in os.listdir(data_folder) if f.endswith('.csv')]

df_year = []
for csv_file in csv_files:
    file_path = os.path.join(data_folder, csv_file)
    df_temp = pd.read_csv(file_path, encoding="cp949")
    df_year.append(df_temp)

df = pd.concat(df_year)
df.shape

In [ ]:
df

In [ ]:
df = df.sort_values(by=['연도', '월', '일', '시']).reset_index(drop=True)
df.shape

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
# 연도별 전력부하합계 변화
plt.figure(figsize=(8, 5))
df.groupby('연도')['전력부하합계'].mean().plot(marker='o')
plt.title('연도별 평균 전력부하합계')
plt.ylabel('평균 전력부하합계')
plt.xlabel('연도')
plt.grid(True)


In [ ]:
plt.figure(figsize=(8,5))
df.groupby('월')['전력부하합계'].mean().plot(marker='o', title='월별 평균 전력부하합계')
plt.show()


In [ ]:
# 시간대별 전력부하합계 변화
plt.figure(figsize=(8, 5))
df.groupby('시')['전력부하합계'].mean().plot(marker='o')
plt.title('시간대별 평균 전력부하합계')
plt.ylabel('평균 전력부하합계')
plt.xlabel('시')
plt.grid(True)
plt.show()


In [ ]:

# 기온, 습도, 풍속과 전력부하합계의 관계 산점도
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
sns.scatterplot(x='기온', y='전력부하합계', data=df.sample(10000), ax=axes[0], alpha=0.3)
axes[0].set_title('기온 vs 전력부하합계')
sns.scatterplot(x='상대습도', y='전력부하합계', data=df.sample(10000), ax=axes[1], alpha=0.3)
axes[1].set_title('상대습도 vs 전력부하합계')
sns.scatterplot(x='풍속', y='전력부하합계', data=df.sample(10000), ax=axes[2], alpha=0.3)
axes[2].set_title('풍속 vs 전력부하합계')
plt.tight_layout()
plt.show()


In [ ]:
corr = df.corr()
corr

In [ ]:
sns.heatmap(corr, cmap='seismic', mask=np.triu(np.ones_like(corr)))

In [ ]:
df.isnull().sum()

In [16]:
# 결측치 처리 (예시: 평균값 대체)
df['상대습도'] = df['상대습도'].fillna(df['상대습도'].mean())
df['풍속'] = df['풍속'].fillna(df['풍속'].mean())

# 특성 공학
# 시간 순환 특성
df['시간_sin'] = np.sin(2 * np.pi * df['시'] / 24)
df['시간_cos'] = np.cos(2 * np.pi * df['시'] / 24)
df['월_sin'] = np.sin(2 * np.pi * df['월'] / 12)
df['월_cos'] = np.cos(2 * np.pi * df['월'] / 12)

# 주말 여부
df['날짜'] = df.apply(lambda x: datetime.datetime(int(x['연도']), int(x['월']), int(x['일'])), axis=1)
df['요일'] = df['날짜'].dt.dayofweek
df['주말'] = df['요일'].apply(lambda x: 1 if x >= 5 else 0)

# 계절 더미 변수
df['여름'] = df['월'].apply(lambda x: 1 if x in [6, 7, 8] else 0)
df['겨울'] = df['월'].apply(lambda x: 1 if x in [12, 1, 2] else 0)

# 이용률, 전력밀도
df['이용률'] = df['전력부하합계'] / df['계약전력합계']
df['전력밀도'] = df['전력부하합계'] / df['공동주택수']

# 냉방/난방도일, 쾌적지수 (예시)
df['냉방도일'] = df['기온'].apply(lambda x: max(0, x - 24))
df['난방도일'] = df['기온'].apply(lambda x: max(0, 18 - x))
df['온도쾌적지수'] = df['기온'] * df['상대습도'] / 100

## 모델 학습시간 단축을 위해 최근 연도만 가져옴

In [ ]:
df = df[df["연도"] == 2022].sample(1000)
df.shape

In [ ]:
df.hist(figsize=(15, 15), bins=50);

In [19]:
# 결측치 처리 (예시: 평균값 대체)
df['상대습도'] = df['상대습도'].fillna(df['상대습도'].mean())
df['풍속'] = df['풍속'].fillna(df['풍속'].mean())
df['기온'] = df['기온'].fillna(df['풍속'].mean())

In [ ]:
df.isnull().sum()

In [ ]:
# 데이터 분할
X = df.drop(['전력부하합계', '날짜', '요일'], axis=1)
y = df['전력부하합계']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [22]:
# 수치형 변수만 선택 (범주형/더미는 상황에 따라)
feaure_names = X.columns
X_train_num = X_train[feaure_names]
X_test_num = X_test[feaure_names]

# 표준화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_num)
X_test_scaled = scaler.transform(X_test_num)

## 모델 훈련 및 예측

In [ ]:
# 모델링 및 평가
models = {
    # 'LinearRegression': LinearRegression(),
    # 'Ridge': Ridge(),
    'RandomForest': RandomForestRegressor(n_estimators=100, random_state=42)
}

for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    print(f"{name}")
    print(f"R2: {r2_score(y_test, y_pred):.4f}")
    print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):.2f}")
    print(f"MAPE: {mean_absolute_percentage_error(y_test, y_pred) * 100:.2f}%")
    print("---")

## 시계열 검증

In [ ]:
# 시계열 검증 (TimeSeriesSplit 예시)
tscv = TimeSeriesSplit(n_splits=5)
for name, model in models.items():
    scores = cross_val_score(model, X_train_scaled, y_train, cv=tscv, scoring='r2')
    print(f"{name} TimeSeries CV R2: {np.mean(scores):.4f} ± {np.std(scores):.4f}")

## 특성 중요도

In [ ]:
# 특성 중요도 (RandomForest)
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train_scaled, y_train)
importances = pd.Series(rf.feature_importances_, index=feaure_names)
print(importances.sort_values(ascending=False))

In [ ]:
plt.figure(figsize=(10, 6))
importances.sort_values(ascending=True).plot(kind='barh')
plt.title('Feature Importances (RandomForest)')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.tight_layout()
plt.show()
